In [1]:
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformer_onnx import save_model_and_config, load_model_and_config, train_transformer, validate_transformer, perform_inference
from transformer_onnx import transform_predicted_to_binary_matrix, Transformer

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import math
import time 
# from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import gurobipy as gp
from gurobipy import GRB
import torch.onnx
import onnxruntime as ort


MPS available, current device MPS


In [2]:
from Test_ONNX_backup import load_data_npz, normalize_batch, normalize_tensor

file_path = "path_to_microgrid_data"

cbuy_tensor, csell_tensor, cprod_tensor, power_res_tensor, power_load_tensor, x0_tensor, delta_transformed_tensor = load_data_npz(file_path)
net_power_load = power_load_tensor - power_res_tensor
src_data = torch.cat([
    cbuy_tensor.unsqueeze(-1),  # Adding an extra dimension for feature alignment
    csell_tensor.unsqueeze(-1),
    cprod_tensor.unsqueeze(-1),
    net_power_load.unsqueeze(-1),
    x0_tensor.unsqueeze(1).repeat(1, cbuy_tensor.size(1), 1)  # Repeating x0 across the sequence length
], dim=-1)  # Concatenate along the last dimension to combine features


#Create target data 
tgt_data = delta_transformed_tensor.long()

print("src_data shape" , src_data.shape)
print("tgt_data shape" , tgt_data.shape)



src_data shape torch.Size([16000, 25, 5])
tgt_data shape torch.Size([16000, 25])


In [3]:
mini_batch_size = 32 
dataset_size = src_data.size(0)
val_size = int(dataset_size * 0.2) 
train_size = dataset_size - val_size

full_dataset = TensorDataset(src_data, tgt_data)
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=mini_batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=mini_batch_size, shuffle=False)

In [4]:
from transformer_onnx import evaluate_model_over_dataset_w_multiple_backups2
transformer, config = load_model_and_config('trained_transformer2.pth')
shallow, config_shallow  = load_model_and_config('shallow_transformer.pth')

models =  ['name_of_transformer_models]
num_instances = 1
evaluate_model_over_dataset_w_multiple_backups2(models, val_dataset, num_instances)



In [ ]:
from transformer_onnx import export_model_to_onnx, normalize_tensor, global_mean, global_std
src_batch, tgt_batch = next(iter(val_dataloader))  # Get a batch of data
src_tensor_normalized = normalize_tensor(src_batch[0:1], global_mean, global_std)  # Normalize and use the first item as an example

# Use a realistic target tensor from your data
real_tgt_tensor = tgt_batch[0:1]  # Use the first target sequence from the batch
print(real_tgt_tensor.shape)
export_model_to_onnx(shallow, src_tensor_normalized, real_tgt_tensor, "shallow")
export_model_to_onnx(shallow_model2, src_tensor_normalized, real_tgt_tensor, "shallow2")
export_model_to_onnx(shallow_model3, src_tensor_normalized, real_tgt_tensor, "shallow3")
export_model_to_onnx(backup_model4, src_tensor_normalized, real_tgt_tensor, "backup4")









In [ ]:
from transformer_onnx import run_onnx_inference, convert_logits_to_tokens

src_batch, tgt_batch = next(iter(val_dataloader))  # Get a batch of data
src_tensor_normalized = normalize_tensor(src_batch[0:1], global_mean, global_std)  # Normalize and use the first item as an example
# Use a realistic target tensor from  data
real_tgt_tensor = tgt_batch[0:1]  # Use the first target sequence from the batch
logits = run_onnx_inference("shallow.onnx", src_tensor_normalized, real_tgt_tensor)

predicted_sequence_onnx = convert_logits_to_tokens(logits)
print("onnx predicted sequence",predicted_sequence_onnx)


predicted_sequence = perform_inference(shallow, src_tensor_normalized)
print(predicted_sequence)


In [ ]:
from transformer_onnx import cascaded_transformer

models =  ['name_transformers']
num_instances = 1
cascaded_transformer(models, val_dataset, num_instances)


In [ ]:
from transformer_onnx import evaluate_models

models_onnx = ["name_exported_onnx_transformers"]
models = [shallow, shallow_model2, shallow_model3, backup_model4]
num_instances = 1000
evaluate_models(models, models_onnx, val_dataset, num_instances) 

